In [8]:
import math
import torch
import random

### We start by building the Value object, to represent the individual components of a Neuron

In [9]:
class Value:
    
    def __init__(self, data, inputs = (), op = "", label = ""):
        self.data = data
        self.predecesors = set(inputs)
        self.op = op
        self.grad = 0
        self.label = label
        self.backward = lambda: None
        
    def __repr__(self) -> str:
        return (f"data of {self.label} = {self.data}; gradient = {self.grad}")
    
    def __radd__(self, other):
        return self + other
    
    def __add__(self, other):
        # check if other is a data:
        other = other if isinstance(other, Value) else Value(other)
        
        # calculate the gradient for addition
        def backward():
            self.grad += out.grad
            other.grad += out.grad
        
        # calculate the output of operation and assign it's local gradient 
        out =  Value(data = self.data + other.data, inputs=(self, other), op = "+")
        out.backward = backward
        
        #return the value
        return out
    
    def __sub__(self, other):
        return self + (-other)
    
    def __rsub__(self, other):
        return self - other
    
    def __rmul__(self, other): # called on other * self, when self is not a number
        return self * other
    
    def __mul__(self, other):
        # check if other is a Value
        other = other if isinstance(other, Value) else Value(other)
        
        # calculate the _gradient for multiplication
        def backward():
            self.grad += out.grad * other.data
            other.grad += out.grad * self.data
        
        # calculate the outpout of the operation and assign it it's respective backward function
        out = Value(data = self.data * other.data, inputs=(self, other), op = "*")
        out.backward = backward
        
        # return the output
        return out
    
    def __truediv__(self, other): # self / other
        return self * other**(-1)
    
    def __rtruediv__(self, other):
        return self * other**(-1)
    
    def __pow__(self, other):
        assert isinstance(other, (float, int)) # this function only work with other
        
        out = Value(data = pow(self.data, other), inputs=(self,), op = "**")
        
        def backward():
            self.grad += other * pow(self.data, other-1) * out.grad
        
        out.backward = backward

        return out
    
    def exp(self):
        x = self.data
        out = Value(data = math.exp(x),inputs=(self,), op="exp")
        def backward():
            self.grad += out.data * out.grad
        out.backward = backward
        return out
    
    def tanh(self):
        x = self.data
        e = math.exp(2*x)
        out = Value(
            data=((e-1)/(e+1)),
            inputs=(self,),
            op="tanh",
        )
        
        def backward():
            self.grad += (1 - out.data**2) * out.grad
    
        out.backward = backward
        
        return out
    
    def backPropagation(self):
        '''
        Start the backpropagation from this node
        '''
        
        # set the gradient of this node to 1
        self.grad = 1
        
        # build a helper function to create the topological sorting of the graph
        def topologicalSorting(node):
            # create local variable
            L = []
            s = set()
            
            # recursive sorting
            def topo(node:Value):
                if node not in s:
                    s.add(node)
                    for parent in node.predecesors:
                        topo(parent)
                    L.append(node)
            topo(node)
            return L
        
        # call the backward method on each node, starting from the last one
        for node in topologicalSorting(self)[::-1]:
            node.backward()
        

### Now we can create the neuron class to build neural networks

In [29]:
class Neuron:
    
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
        
    def __call__(self, x):
        activation = sum((wi*xi for wi, xi in zip(self.w, x)),self.b).tanh()
        return activation
    
class Layer:
    
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self, x):
        activation = [n(x) for n in self.neurons]
        return activation
    
class MLP:
    
    def buildLayers(nin, nouts):
        layers = [Layer(nin, nouts[0])]
        for n in range(1,len(nouts)):
            layers.append(Layer(nouts[n-1], nouts[n]))
        return layers
    
    def __init__(self, nin, nouts:list):
        self.io = [nin] + nouts
        self.layers = [Layer(self.io[n], self.io[n+1]) for n in range(len(nouts))]
    
    def __call__(self, x):
        activation = [layer(x) for layer in self.layers]
        return activation


In [30]:
# test activation for a neuron
neuron = Neuron(3)
neuron((1,2,3))

# test activation for a layer
layer = Layer(3,4)
layer((1,2,3))

# test activation for a neural network
nn = MLP(3,[4,4])
nn((1,2,3))

[[data of  = 0.8969144768112074; gradient = 0,
  data of  = 0.005159696092638309; gradient = 0,
  data of  = 0.4672414160402595; gradient = 0,
  data of  = 0.9823025046346043; gradient = 0],
 [data of  = -0.9978851657256539; gradient = 0,
  data of  = 0.9992265756136209; gradient = 0,
  data of  = -0.9929183651806309; gradient = 0,
  data of  = -0.9239658678427609; gradient = 0]]